In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

total_epoch = 100
batch_size = 100
learning_rate = 0.0002
n_hidden = 256
n_input = 28 * 28
# 노이즈의 크기로 랜덤한 노이즈 입력하고 그 노이즈에서 손글씨 이미지를 무작위로 생성해내도록 할 변수
n_noise = 128
-
# X는 실제이미지를 넣을 변수 Z는 노이즈에서 생성된 가짜이미지를 넣을 변수
X = tf.placeholder(tf.float32, [None, n_input])
Z = tf.placeholder(tf.float32, [None, n_noise])

#생성자 신경망에 사용할 변수 설정
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

#구분자 신경망에 사용할 변수 설정
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))

# 생성자, 구분자 신경망 구성
# 생성자신경망
def generator(noise_z):
    hidden = tf.nn.relu(
                    tf.matmul(noise_z, G_W1) + G_b1)
    output = tf.nn.sigmoid(
                    tf.matmul(hidden, G_W2) + G_b2)
    return output
                   
# 구분자신경망
def discriminator(inputs):
    hidden = tf.nn.relu(
                    tf.matmul(inputs, D_W1) + D_b1)
    output = tf.nn.sigmoid(
                    tf.matmul(hidden, D_W2) + D_b2)
    return output

# 무작위 노이즈 생성해주는 함수
def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size, n_noise))

# 노이즈 Z를 이용해 가짜 이미지를 만들 생성자 G를 만듦
# G가 만든 가짜 이미지와 진짜 이미지를 각각 구분자에 넣어 입력한 이미지가 진짜 인지 판별 하도록 함.
G = generator(Z)
D_gene = discriminator(G)
D_real = discriminator(X)

# 손실값
# 두개가 필요함. 경찰 학습용 손실값과, 위조지폐범 학습용을 구함.
# 경찰 학습용 : 생성자가 만든 이미지를 구분자가 가짜라고 판단하도록 하는 손실값
# D_gene는 가짜를 판별하는 값이므로 0에 가까워야 함.(가짜라고 판별)
# D_real은 진짜 이미지 판별값이므로 1에 가까워야 함.(진짜라고 판별)
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1 - D_gene))


# 범인 학습용 : 생성자가 만든 이미지를 구분자가 진짜라고 판단 하도록 하는 손실값.
# D_gene를 1에 가깝게 만들기만 하면됨.
# 즉 가짜 이미지를 넣어도 진짜같다고 판별해야 함. D_gene값을 최대화 하면 위조지폐범을 학습 시킬 수 있음.
loss_G = tf.reduce_mean(tf.log(D_gene))

# 결과적으로 GAN의 목표는 loss_D와 loss_G를 모두 최대화 하는 것
# 하지만 서로 반비례 관계(경쟁관계)에 있기때문에 항상 같이 증가하는 경향을 보이지는 않음.

# 손실값을 이용해 학습
# loss_D를 구할 때는 구분자 신경망에 사용되는 변수들만 사용.
# loss_G를 구할 때는 생성자 신경망에 사용되는 변수들만 사용.
# 그래야 각각 학습 시켰을때 신경망의 변화가 생기지 않는다.
D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]

# 최적화 함수 구성
# GAN의 목표는 loss_D, loss_G의 값이 최대값이 되는 것.
# 최적화함수는 minimize뿐이므로 "-" 를 붙여줌.
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D, var_list=D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G, var_list=G_var_list)

##########
#학습 코드
##########
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples / batch_size)
loss_val_D, loss_val_G = 0, 0

for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        
        _, loss_val_D = sess.run([train_D, loss_D], feed_dict={X: batch_xs, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G], feed_dict={Z: noise})
    
    print('Epoch:', '%04d' % epoch, 
          'D loss: {:.4}'.format(loss_val_D),
          'G loss: {:.4}'.format(loss_val_G))
    
    ##############
    #학습결과 확인
    ##############
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict={Z: noise})
        
        fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1))
        
        for i in range(sample_size):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i], (28, 28)))
            
        plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)),
                    bbox_inches='tight')
        plt.close(fig)
        
print('최적화 완료!')
    

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Epoch: 0000 D loss: -0.4567 G loss: -2.094
최적화 완료!
Epoch: 0001 D loss: -0.5359 G loss: -1.993
최적화 완료!
Epoch: 0002 D loss: -0.1828 G loss: -2.717
최적화 완료!
Epoch: 0003 D loss: -0.4081 G loss: -1.727
최적화 완료!
Epoch: 0004 D loss: -0.2019 G loss: -2.362
최적화 완료!
Epoch: 0005 D loss: -0.2399 G loss: -2.808
최적화 완료!
Epoch: 0006 D loss: -0.1626 G loss: -3.58
최적화 완료!
Epoch: 0007 D loss: -0.1849 G loss: -2.831
최적화 완료!
Epoch: 0008 D loss: -0.26 G loss: -2.594
최적화 완료!
Epoch: 0009 D loss: -0.1482 G loss: -3.103
최적화 완료!
Epoch: 0010 D loss: -0.2872 G loss: -2.904
최적화 완료!
Epoch: 0011 D loss: -0.2947 G loss: -2.682
최적화 완료!
Epoch: 0012 D loss: -0.2806 G loss: -3.137
최적화 완료!
Epoch: 0013 D loss: -0.353 G loss: -2.684
최적화 완료!
Epoch: 0014 D loss: -0.222 G loss: -2.989
최적화 완료!
Epoch: 0015 D loss: -0.4115 G loss: -2